In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.backend import mean, square

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GlobalAttentionPool
from spektral.utils import label_to_one_hot

from os import path
import itertools

In [ ]:
A_complete, X_complete, E_complete, y_complete = qm9.load_data(return_type='numpy',
                           nf_keys='atomic_num',
                           ef_keys='type',
                           self_loops=True,
                           amount=None)  # Set to None to train on whole dataset
# one-hot labeling of atoms
uniq_X = np.unique(X_complete)
X_complete = label_to_one_hot(X_complete, uniq_X)

In [ ]:
A, X, E = list(), list(), list()
y = y_complete.sample(10000)
for index, row in y.iterrows():
    A.append(A_complete[index])
    X.append(X_complete[index])
    E.append(E_complete[index])
A = np.stack(A, axis=0)
X = np.stack(X, axis=0)
E = np.stack(E, axis=0)

In [ ]:
tasks = list(y.columns)[1:]
y_list = []
for task in tasks:
    y_list.append(y[[task]].values)
for i in range(len(y_list)):
    y_list[i] = StandardScaler().fit_transform(y_list[i]).reshape(-1, y_list[0].shape[-1])

In [ ]:
N = X.shape[-2]
F = X.shape[-1] 
S = E.shape[-1]
n_out = y_list[0].shape[-1]
learning_rate = 1e-3
epochs = 10
batch_size = 64
es_patience = 5

In [ ]:
n_out

In [ ]:
optimizer = Adam(lr=learning_rate)
loss = 'mse'

In [ ]:
A_train, A_test, \
X_train, X_test, \
E_train, E_test, \
*y_train_test_list = train_test_split(A, X, E, *y_list, test_size = 0.1)

y_train_list = y_train_test_list[::2]
y_test_list = y_train_test_list[1::2]

In [ ]:
X_in = Input(shape=(N, F))
A_in = Input(shape=(N, N))
E_in = Input(shape=(N, N, S))

In [ ]:
def create_single_task_model(X_in, A_in, E_in):
    gc1 = EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in])
    gc2 = EdgeConditionedConv(128, activation='relu')([gc1, A_in, E_in])
    pool = GlobalAttentionPool(256)(gc2)
    dense1 = Dense(256, activation='relu')(pool)
    output = Dense(n_out)(dense1)
    return Model(inputs=[X_in, A_in, E_in], outputs=output)

In [ ]:
def generate_filename(task):
    return path.join('gcnmodels', task + '.h5')

In [ ]:
print(tasks)

In [ ]:
for i in range(0, 15):
    if i == 4:
        continue
    if i == 8:
        continue
    print('learning', tasks[i])
    model = create_single_task_model(X_in, A_in, E_in)
    model.compile(optimizer=optimizer, loss=loss)
    es_callback = EarlyStopping(monitor='val_loss', patience=es_patience)
    model.fit([X_train, A_train, E_train],
             y_train_list[i],
             batch_size=batch_size,
             validation_split=0.1,
             epochs=epochs,
             callbacks=[es_callback])
    model.save_weights(generate_filename(tasks[i]))